# Guardamos las credenciales

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["QDRANT_URL"] = ""
os.environ["QDRANT_API_KEY"] = ""

## Instalamos dependencias

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-qdrant
!pip install os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [ ]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## Accedemos a nuestro VectorStore ya creado, que tiene cargado los documentos



In [ ]:
url = os.environ["QDRANT_URL"]
api_key = os.environ["QDRANT_API_KEY"]
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    url=url,
    api_key=api_key,
    collection_name="mi_primer_rag",

)

## Buscar con y sin score de similaridad

In [ ]:
results = qdrant.similarity_search(
    "La giralda", k=1
)
#print(results)
for res in results:
    print(f"* CONTENIDO{res.page_content} METADATA:{res.metadata} {res}")

* CONTENIDO/
Guía Práctica de Sevilla
QUÉ VISITAR
21
MONUMENTOS  
PATRIMONIO
DE LA HUMANIDAD,
SANTA CRUZ Y 
SAN BARTOLOMÉ
ZONA 1/
La Catedral de Santa María de la Sede es el mayor templo 
gótico del mundo y el tercer templo de la Cristiandad, tras 
San Pedro del Vaticano y San Pablo de Londres. Construida 
sobre el lugar que ocupaba la principal mezquita almohade, 
las obras se comenzaron en el siglo XV. De la antigua 
mezquita se conservan aún restos, como su alminar, la 
Giralda, símbolo de la ciudad, y el patio de los Naranjos, 
patio de abluciones. 
En la Catedral trabajaron algunos de los principales artistas 
de la época, y durante su proceso constructivo se añadieron 
elementos renacentistas, barrocos y neoclásicos. Sus 
numerosas capillas guardan magníficas obras de arte, como 
cuadros de Murillo, Goya o Zurbarán; esculturas de Martínez 
Montañés, Juan de Mesa o Alonso Cano; obras de orfebrería 
como la urna del Rey San Fernando, realizada por Juan 
Laureano de Pina y hermosas 

In [ ]:
results = qdrant.similarity_search_with_score(
    "Oso panda", k=1
)
print(results)

[(Document(metadata={'page_label': '14', 'city': 'Malaga', 'source': 'https://www.spain.info/export/sites/segtur/.content/Folletos/folletos/Malaga_ES.pdf', 'page': 13, '_id': '6705751a-3b79-4eb4-b345-58bbd9a58943', '_collection_name': 'mi_primer_rag'}, page_content='14\nANTEQUERA\nFamosa por sus dulces navideños ar -\ntesanales, paseando por su centro des -\ncubrirás palacios, conventos, ermitas, \ncasas señoriales y hasta una Alcazaba. \nEncontrarás construcciones de la Edad \nde Bronce, como el Dolmen de la Cueva \nde Menga.\nMálaga presume de ser una de las pro -\nvincias más montañosas de España. \nPractica el senderismo en entornos es -\npectaculares como el T orcal de Ante -\nquera, un paraje natural con extrañas \nformaciones en roca kárstica.\nEl colorido del invierno malagueño gira \nen\ttorno\ta\tla\tfloración\tdel\talmendro.\tLo-\ncalidades como Casabermeja, Arenas, \nArdales, Carratraca, Almogía, Periana \no Guaro se llenan de tonalidades blan -\ncas y rosáceas.\nSi vienes 

# Usando Qdrant Client

In [ ]:
from qdrant_client import QdrantClient

In [ ]:
qdrant_client = QdrantClient(
  url = os.environ["QDRANT_URL"],
  api_key = os.environ["QDRANT_API_KEY"]
)

In [ ]:
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name= "mi_primer_rag",
    embedding=embeddings,
)

In [ ]:
query = "La giralda"
query_vector = embeddings.embed_query(query)

In [ ]:
results = qdrant_client.search(
    collection_name="mi_primer_rag",
    query_vector=query_vector,
    limit=1,  # Número de resultados a devolver
)

# Imprimir resultados
for i, result in enumerate(results):
    print(f"Resultado {i + 1}:")
    print(f"* CONTENIDO: {result.payload.get('page_content')}")
    print(f"* METADATA: {result.payload.get('metadata', 'No disponible')}")
    print(f"* SCORE: {result.score}")

Resultado 1:
* CONTENIDO: /
Guía Práctica de Sevilla
QUÉ VISITAR
21
MONUMENTOS  
PATRIMONIO
DE LA HUMANIDAD,
SANTA CRUZ Y 
SAN BARTOLOMÉ
ZONA 1/
La Catedral de Santa María de la Sede es el mayor templo 
gótico del mundo y el tercer templo de la Cristiandad, tras 
San Pedro del Vaticano y San Pablo de Londres. Construida 
sobre el lugar que ocupaba la principal mezquita almohade, 
las obras se comenzaron en el siglo XV. De la antigua 
mezquita se conservan aún restos, como su alminar, la 
Giralda, símbolo de la ciudad, y el patio de los Naranjos, 
patio de abluciones. 
En la Catedral trabajaron algunos de los principales artistas 
de la época, y durante su proceso constructivo se añadieron 
elementos renacentistas, barrocos y neoclásicos. Sus 
numerosas capillas guardan magníficas obras de arte, como 
cuadros de Murillo, Goya o Zurbarán; esculturas de Martínez 
Montañés, Juan de Mesa o Alonso Cano; obras de orfebrería 
como la urna del Rey San Fernando, realizada por Juan 
Laureano de P

<ipython-input-12-b6ff5c405482>:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


# Filtrando por Metadata



## Definimos el filtro -> https://qdrant.tech/documentation/concepts/filtering/

In [ ]:
from qdrant_client.http import models

search_filter = models.Filter(
    should=[
        models.FieldCondition(
           key="metadata.city", match={"value": "Malaga"}
        ),
    ]
)

In [ ]:
results = vector_store.similarity_search(
    query="Bares",
    k =3,  # Número de resultados a devolver
    filter=search_filter  # Aplicar el filtro
)

# Imprimir resultados
for i, result in enumerate(results):
    print(f"Resultado {i + 1}:")
    print(f"* CONTENIDO: {result}")

Resultado 1:
* CONTENIDO: page_content='4
Recorre la ciudad de Málaga y descubre la personalidad de cada uno de sus barrios. 
Estos son los imprescindibles. 
CENTRO HISTÓRICO
Pasea por sus animadas calles peatona -
les, repletas de bares y restaurantes. La 
plaza de la Constitución es centro neu -
rálgico de la vida malagueña. Cerca en -
contrarás la Iglesia de los Santos Márti-
res (siglos XV-XVIII).
En el casco antiguo te sorprenderá la in-
acabada torre derecha de la Catedral (si-
glos XVI-XVIII), el Palacio Episcopal (siglos 
XVI-XVIII) y Centro Cultural Fundación 
Unicaja Málaga, un espacio dedicado a ex-
posiciones artísticas. A los pies del monte 
Gibralfaro descubrirás el T eatro Romano 
(siglo I a. C.) y la Alcazaba (siglo XI), una 
fortaleza árabe símbolo de la ciudad. 
Si te gustan las compras, en la calle del 
Marqués de Larios tienes las principales 
marcas de moda.
4
VIVE MÁLAGA  
BARRIO A BARRIO
MÁLAGA
 a CALLE DEL MARQUÉS DE LARIOS
 a PALACIO EPISCOPAL' metadata={'city'